# Concurrency Management in Python

This notebook introduces concurrency concepts in Python, including threading, multiprocessing, and async programming. You'll learn when and how to use each approach, with practical code examples.

## Table of Contents
1. Introduction to Concurrency
2. Threading
3. Multiprocessing
4. AsyncIO (Asynchronous Programming)
5. When to Use Each Approach
6. Best Practices

## 1. Introduction to Concurrency

Concurrency allows a program to manage multiple tasks at the same time, improving efficiency and responsiveness. Python supports several concurrency models:
- Threading: Lightweight, shared memory, good for I/O-bound tasks.
- Multiprocessing: Separate processes, good for CPU-bound tasks.
- AsyncIO: Single-threaded, event-driven, good for high-level structured network code.

## 2. Threading

Threading enables concurrent execution of code using multiple threads within a single process. Useful for I/O-bound operations (e.g., file/network access).

In [1]:
import threading
import time

def print_numbers():
    for i in range(5):
        print(f"Number: {i}")
        time.sleep(0.5)

def print_letters():
    for letter in 'abcde':
        print(f"Letter: {letter}")
        time.sleep(0.5)

# Create threads
thread1 = threading.Thread(target=print_numbers)
thread2 = threading.Thread(target=print_letters)

# Start threads
thread1.start()
thread2.start()

# Wait for both threads to finish
thread1.join()
thread2.join()
print("Threading example complete.")

Number: 0
Letter: a
Number: 1
Letter: b
Number: 2
Letter: c
Number: 3
Letter: d
Number: 4
Letter: e
Threading example complete.


## 3. Multiprocessing

Multiprocessing allows you to run code in parallel using multiple processes, each with its own Python interpreter. Useful for CPU-bound tasks.

In [2]:
from multiprocessing import Process, current_process

def square_numbers():
    for i in range(5):
        print(f"{current_process().name} squared: {i*i}")

if __name__ == "__main__":
    p1 = Process(target=square_numbers, name="Process-1")
    p2 = Process(target=square_numbers, name="Process-2")
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    print("Multiprocessing example complete.")

Process-1 squared: 0
Process-1 squared: 1
Process-2 squared: 0Process-1 squared: 4

Process-1 squared: 9Process-2 squared: 1

Process-1 squared: 16Process-2 squared: 4



Process-2 squared: 9
Process-2 squared: 16
Multiprocessing example complete.


## 4. AsyncIO (Asynchronous Programming)

AsyncIO is a library to write concurrent code using the async/await syntax. It is ideal for I/O-bound and high-level structured network code.

In [3]:
import asyncio

async def async_task(name, delay):
    print(f"Task {name} started")
    await asyncio.sleep(delay)
    print(f"Task {name} finished after {delay} seconds")

async def main():
    await asyncio.gather(
        async_task("A", 1),
        async_task("B", 2),
        async_task("C", 1)
    )

await main()

Task A started
Task B started
Task C started
Task A finished after 1 seconds
Task C finished after 1 seconds
Task B finished after 2 seconds


## 5. When to Use Each Approach

- **Threading:** Use for I/O-bound tasks (e.g., web requests, file I/O).
- **Multiprocessing:** Use for CPU-bound tasks (e.g., data processing, computation).
- **AsyncIO:** Use for high-level I/O-bound and network code, especially when you need to handle many connections efficiently.

## 6. Best Practices
- Avoid shared state in multiprocessing; use queues or pipes for communication.
- Use locks or other synchronization primitives to avoid race conditions in threading.
- Prefer async for scalable I/O-bound applications.
- Profile your code to choose the right concurrency model.

## 2.1 ThreadPoolExecutor with concurrent.futures

The `concurrent.futures.ThreadPoolExecutor` provides a high-level interface for asynchronously executing callables using threads. It is often easier and safer to use than manually managing threads.

In [4]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def task(name, delay):
    print(f"Task {name} started")
    time.sleep(delay)
    print(f"Task {name} finished after {delay} seconds")
    return name

with ThreadPoolExecutor(max_workers=3) as executor:
    futures = [executor.submit(task, f"T{i}", i) for i in range(1, 4)]
    for future in as_completed(futures):
        result = future.result()
        print(f"Result: {result}")

Task T1 started
Task T2 started
Task T3 started
Task T1 finished after 1 seconds
Result: T1
Task T2 finished after 2 seconds
Result: T2
Task T3 finished after 3 seconds
Result: T3


**When to use ThreadPoolExecutor:**
- When you want to run many I/O-bound tasks concurrently with a simple interface.
- It manages thread creation, scheduling, and joining for you.

## 2.2 ProcessPoolExecutor with concurrent.futures

The `concurrent.futures.ProcessPoolExecutor` provides a high-level interface for parallel execution using separate processes. It is ideal for CPU-bound tasks and avoids the Global Interpreter Lock (GIL).

In [5]:
from concurrent.futures import ProcessPoolExecutor, as_completed

def compute_square(n):
    return n * n

with ProcessPoolExecutor(max_workers=2) as executor:
    numbers = [1, 2, 3, 4, 5]
    futures = [executor.submit(compute_square, num) for num in numbers]
    for future in as_completed(futures):
        print(f"Result: {future.result()}")

Result: 4
Result: 1
Result: 9
Result: 16
Result: 25


**When to use ProcessPoolExecutor:**
- For CPU-bound tasks that benefit from true parallelism.
- When you want a simple interface for running functions in separate processes.